In [1]:
!pip install neuralforecast

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.6/170.6 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.9/776.9 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.6/230.6 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.0 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import numpy as np
import torch
from typing import Optional, Union, Tuple


import torch.nn.functional as F
from torch.distributions import Distribution
from torch.distributions import Bernoulli, Normal, StudentT, Poisson, NegativeBinomial

from torch.distributions import constraints


from neuralforecast import NeuralForecast
from neuralforecast.models import NHITS
from neuralforecast.tsdataset import TimeSeriesDataset
from neuralforecast.utils import AirPassengers, AirPassengersPanel, AirPassengersStatic
from neuralforecast.losses.numpy import rmse, mape
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
from neuralforecast.losses.pytorch import MQLoss, DistributionLoss, MSE, MAE

In [3]:
from numpy.random import seed
from random import randrange

In [4]:
df = pd.read_excel("SP100_vol (clean).xlsx", index_col="Date",parse_dates=True)
columns=df.columns
df_static=pd.read_excel("Static_df.xlsx")
n_series=len(df.columns)

In [5]:
n_inputs = [3,5,10,15,21,42,84]
mlp_units = [[[712, 712], [712, 712]],[[512, 512], [512, 512]],[[250, 250], [250, 250]],[[100, 100], [100, 100]]]
epochs=[50,100,150,250,350,450,550,650,750]
learning_rate=[0.0005,0.0001,0.00005,0.00001]
num_lr_decays=[5,3,2,1]
scaler_type=["robust","standard",'minmax']
losses=[MSE(),MAE(),MQLoss(level=[80, 90]),DistributionLoss(distribution='StudentT', level=[80, 90])]
n_blocks=[[1, 1],[2, 2],[3, 3],[5, 5]]
dropout_prob_theta=[0, 0.2, 0.3, 0.4]
n_pool_kernel_size=[[2, 2, 1], [4, 2, 1], [4, 4, 2]]
pooling_mode=["MaxPool1d", "AvgPool1d"]
interpolation_mode=['linear', 'nearest']

# Test

In [6]:
test_length=int(len(df['NKE'])*0.3)
df1=df.unstack().reset_index(name="Value").rename(columns={"level_0":"unique_id", "Date":"ds", "Value":"y"})

In [8]:
RMSE = []
QLIKE = []
MAE = []
MAPE = []
H=5

for m in range (20):
  randomseed=randrange(129228148)
  model = NHITS(h=H, input_size=21,
                  loss=losses[3],
                  scaler_type='standard',
                  learning_rate=learning_rate[0],
                  stack_types=['identity','identity'],
                  n_blocks=[2,2],
                  mlp_units= [[712, 712], [712, 712]],
                  windows_batch_size=32,
                  num_lr_decays=num_lr_decays[0],
                  val_check_steps=500,
                  stat_exog_list=['AMT', 'COP', 'CVX', 'AXP', 'BAC', 'C', 'CB', 'CME', 'GS', 'JPM', 'MS',
         'PNC', 'USB', 'WFC', 'BA', 'CAT', 'GE', 'HON', 'LMT', 'MMM', 'UNP',
         'UPS', 'AAPL', 'ACN', 'ADBE', 'ADP', 'CMCSA', 'CRM', 'CSCO', 'DIS',
         'FIS', 'FISV', 'GOOG', 'IBM', 'INTC', 'INTU', 'MA', 'MSFT', 'NFLX',
         'NVDA', 'ORCL', 'QCOM', 'T', 'TXN', 'VZ', 'COST', 'KO', 'MO', 'PEP',
         'PG', 'WMT', 'D', 'DUK', 'SO', 'ABT', 'AMGN', 'BDX', 'BMY', 'BSX', 'CI',
         'CVS', 'DHR', 'GILD', 'ISRG', 'JNJ', 'LLY', 'MDT', 'MRK', 'PFE', 'SYK',
         'TMO', 'UNH', 'AMZN', 'HD', 'LOW', 'MCD', 'NKE', 'SBUX', 'TGT', 'TJX'],
                  max_steps=150,
                  dropout_prob_theta = 0,
                  n_pool_kernel_size=[4,2,1],
                  pooling_mode='MaxPool1d' ,
                  interpolation_mode='nearest',
                random_seed=randomseed)
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=df1,val_size=H,static_df=df_static,n_windows=None, test_size=test_length-test_length%H,step_size=H)
  forecasts = forecasts.dropna()
  if "NHITS-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["NHITS"].values
  else:
    Y_hat=forecasts["NHITS-median"].values
  Y_true=forecasts["y"].values
  RMSE.append(np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true)))
  QLIKE.append((np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true)))
  MAE.append(np.sum(abs(Y_true-Y_hat))/len(Y_true))
  MAPE.append(np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true))

print(f"RMSE mean: {np.mean(RMSE)} | QLIKE mean: {np.mean(QLIKE)} | MAE mean: {np.mean(MAE)} | MAPE mean: {np.mean(MAPE)}")
print(f"RMSE std: {np.std(RMSE)} | QLIKE std: {np.std(QLIKE)} | MAE std: {np.std(MAE)} | MAPE std: {np.std(MAPE)}")
from scipy.stats import anderson
print(f"RMSE: {anderson(RMSE)}")
print(f"QLIKE: {anderson(QLIKE)}")
print(f"MAE: {anderson(MAE)}")
print(f"MAPE: {anderson(MAPE)}")

df_errors = pd.DataFrame({
    'RMSE': RMSE,
    'QLIKE': QLIKE,
    'MAE': MAE,
    'MAPE': MAPE
})

df_errors.to_excel("NHITS forecasts.xlsx")

INFO:lightning_fabric.utilities.seed:Seed set to 8720052


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

RMSE mean: 0.0048308283691758745 | QLIKE mean: 0.04600127171806258 | MAE mean: 0.002911466568040489 | MAPE mean: 0.21635916356423257
RMSE std: 4.335958065921434e-05 | QLIKE std: 0.0007830596021289519 | MAE std: 2.2450685311230287e-05 | MAPE std: 0.003557473418143033
RMSE: AndersonResult(statistic=0.4163618521690715, critical_values=array([0.506, 0.577, 0.692, 0.807, 0.96 ]), significance_level=array([15. , 10. ,  5. ,  2.5,  1. ]), fit_result=  params: FitParams(loc=0.0048308283691758745, scale=4.448599111184405e-05)
 success: True
 message: '`anderson` successfully fit the distribution to the data.')
QLIKE: AndersonResult(statistic=0.2594084002045278, critical_values=array([0.506, 0.577, 0.692, 0.807, 0.96 ]), significance_level=array([15. , 10. ,  5. ,  2.5,  1. ]), fit_result=  params: FitParams(loc=0.04600127171806258, scale=0.0008034022001767185)
 success: True
 message: '`anderson` successfully fit the distribution to the data.')
MAE: AndersonResult(statistic=0.33320010759749863,